In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import os, random, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb
from kaggle_secrets import UserSecretsClient

# W&B login via Kaggle secrets
user_secrets = UserSecretsClient()
wandb.login(key=user_secrets.get_secret("wandb-key"))

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(NCI1)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class CentralityEncoder(nn.Module):
    def __init__(self, in_dim=1, out_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, out_dim)
        )
    def forward(self, c):
        return self.net(c)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = CentralityEncoder(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent, edge_index, batch):
        x = torch.cat([x, self.cent_enc(cent)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def _minmax_norm(t):
    if len(t) == 0:
        return t
    vmin, vmax = t.min(), t.max()
    if (vmax - vmin) > 0:
        return (t - vmin) / (vmax - vmin)
    return torch.zeros_like(t)

def make_centrality_transform():
    def transform(data):
        if getattr(data, "x", None) is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        G = to_networkx(data, to_undirected=True)
        cent = torch.zeros((data.num_nodes, 1), dtype=torch.float)
        if G.number_of_nodes() > 0:
            try:
                ev = nx.eigenvector_centrality_numpy(G)
            except Exception:
                ev = {n: 0.0 for n in G.nodes}
            ev_tensor = torch.tensor([ev.get(n, 0.0) for n in range(data.num_nodes)], dtype=torch.float)
            cent[:, 0] = _minmax_norm(ev_tensor)
        data.cent = cent
        return data
    return transform

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root="data/TUDataset", name="NCI1", transform=make_centrality_transform())
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GCN(dataset.num_features if dataset.num_features else 1, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Centrality_PROTEINS_seed_{seed}",
        group="GCN_PROTEINS_Centrality",
        job_type="centrality_only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Centrality_eigenvector",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss, correct, count = 0.0, 0, 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.cent, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return total_loss / len(train_loader), correct / count

    def test(loader):
        model.eval()
        correct, count = 0, 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.cent, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return correct / count

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({
            "epoch": epoch,
            "loss": loss,
            "train_acc": train_acc,
            "test_acc": test_acc
        })

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galsalman (noynetanel1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250914_173458-h1ti0dcw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_Centrality_PROTEINS_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: 🚀 View run at https://wandb.ai

Seed 1 | Epoch 001 | Loss: 0.6853 | Train Acc: 0.5416 | Test Acc: 0.5995
Seed 1 | Epoch 002 | Loss: 0.6784 | Train Acc: 0.5611 | Test Acc: 0.5324
Seed 1 | Epoch 003 | Loss: 0.6651 | Train Acc: 0.5864 | Test Acc: 0.5981
Seed 1 | Epoch 004 | Loss: 0.6554 | Train Acc: 0.6117 | Test Acc: 0.5684
Seed 1 | Epoch 005 | Loss: 0.6524 | Train Acc: 0.6161 | Test Acc: 0.6200
Seed 1 | Epoch 006 | Loss: 0.6490 | Train Acc: 0.6287 | Test Acc: 0.6307
Seed 1 | Epoch 007 | Loss: 0.6453 | Train Acc: 0.6326 | Test Acc: 0.6019
Seed 1 | Epoch 008 | Loss: 0.6358 | Train Acc: 0.6472 | Test Acc: 0.5859
Seed 1 | Epoch 009 | Loss: 0.6305 | Train Acc: 0.6589 | Test Acc: 0.6345
Seed 1 | Epoch 010 | Loss: 0.6209 | Train Acc: 0.6657 | Test Acc: 0.6647
Seed 1 | Epoch 011 | Loss: 0.6198 | Train Acc: 0.6613 | Test Acc: 0.6560
Seed 1 | Epoch 012 | Loss: 0.6082 | Train Acc: 0.6822 | Test Acc: 0.6818
Seed 1 | Epoch 013 | Loss: 0.6037 | Train Acc: 0.6832 | Test Acc: 0.6097
Seed 1 | Epoch 014 | Loss: 0.6100 | Train Acc: 0.66

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▆▆▆▅▅▄▄▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁
wandb:  test_acc ▄▁▄▃▅▄▃▅▇▆▄▆▇▇▇▇▇▇▇▇█▇█▇▇▆█▇▇█▅█▆█▇▇▇█▇█
wandb: train_acc ▁▂▄▄▅▆▆▆▇▇▇▇▇▇▇▇█▇▇█▇▇██▇███████████████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.57352
wandb:  test_acc 0.68856
wandb: train_acc 0.69781
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/h1ti0dcw
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_173458-h1ti0dcw/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/workin

Seed 2 | Epoch 001 | Loss: 0.6886 | Train Acc: 0.5460 | Test Acc: 0.5601
Seed 2 | Epoch 002 | Loss: 0.6739 | Train Acc: 0.5898 | Test Acc: 0.6185
Seed 2 | Epoch 003 | Loss: 0.6615 | Train Acc: 0.6054 | Test Acc: 0.6151
Seed 2 | Epoch 004 | Loss: 0.6626 | Train Acc: 0.5971 | Test Acc: 0.6287
Seed 2 | Epoch 005 | Loss: 0.6523 | Train Acc: 0.6302 | Test Acc: 0.5995
Seed 2 | Epoch 006 | Loss: 0.6483 | Train Acc: 0.6311 | Test Acc: 0.6472
Seed 2 | Epoch 007 | Loss: 0.6407 | Train Acc: 0.6511 | Test Acc: 0.6540
Seed 2 | Epoch 008 | Loss: 0.6359 | Train Acc: 0.6511 | Test Acc: 0.6579
Seed 2 | Epoch 009 | Loss: 0.6243 | Train Acc: 0.6676 | Test Acc: 0.6453
Seed 2 | Epoch 010 | Loss: 0.6193 | Train Acc: 0.6764 | Test Acc: 0.6277
Seed 2 | Epoch 011 | Loss: 0.6167 | Train Acc: 0.6686 | Test Acc: 0.6745
Seed 2 | Epoch 012 | Loss: 0.6077 | Train Acc: 0.6842 | Test Acc: 0.6579
Seed 2 | Epoch 013 | Loss: 0.6076 | Train Acc: 0.6861 | Test Acc: 0.6550
Seed 2 | Epoch 014 | Loss: 0.6033 | Train Acc: 0.68

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▇▆▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:  test_acc ▁▄▄▅▃▆▆▅▅▇▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█▇█
wandb: train_acc ▁▃▃▃▄▅▅▆▆▆▇▆▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.57405
wandb:  test_acc 0.691
wandb: train_acc 0.7017
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/y0qpm4uw
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_174045-y0qpm4uw/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/w

...

wandb: updating run metadata
wandb: uploading history steps 48-49, summary, console lines 48-49
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▆▆▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▁▂▁▂▂▂▁▂▁▁▁▁▂
wandb:  test_acc ▁▄▄▅▅▆▇▇▇▇▆▇▇▇▆▅▇▇▇▇▆▇▇▇▇▇██▇▇▇████▇▇▆██
wandb: train_acc ▁▃▃▄▄▅▆▆▆▆▆▆▆▇▆▇▆▆▇▇▇▇▇▇█▇▇▇▇█▇▇█▇█████▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.57347
wandb:  test_acc 0.70511
wandb: train_acc 0.70219
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/jd65ejee
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_200338-jd65ejee/logs

Seed 29 | Epoch 001 | Loss: 0.6862 | Train Acc: 0.5353 | Test Acc: 0.5946
Seed 29 | Epoch 002 | Loss: 0.6672 | Train Acc: 0.6024 | Test Acc: 0.5693
Seed 29 | Epoch 003 | Loss: 0.6616 | Train Acc: 0.6219 | Test Acc: 0.6229
Seed 29 | Epoch 004 | Loss: 0.6570 | Train Acc: 0.6141 | Test Acc: 0.6161
Seed 29 | Epoch 005 | Loss: 0.6545 | Train Acc: 0.6175 | Test Acc: 0.6418
Seed 29 | Epoch 006 | Loss: 0.6474 | Train Acc: 0.6224 | Test Acc: 0.6506
Seed 29 | Epoch 007 | Loss: 0.6406 | Train Acc: 0.6345 | Test Acc: 0.6633
Seed 29 | Epoch 008 | Loss: 0.6363 | Train Acc: 0.6487 | Test Acc: 0.6681
Seed 29 | Epoch 009 | Loss: 0.6310 | Train Acc: 0.6608 | Test Acc: 0.6686
Seed 29 | Epoch 010 | Loss: 0.6284 | Train Acc: 0.6428 | Test Acc: 0.6769
Seed 29 | Epoch 011 | Loss: 0.6210 | Train Acc: 0.6599 | Test Acc: 0.6818
Seed 29 | Epoch 012 | Loss: 0.6169 | Train Acc: 0.6701 | Test Acc: 0.6929
Seed 29 | Epoch 013 | Loss: 0.6209 | Train Acc: 0.6764 | Test Acc: 0.6769
Seed 29 | Epoch 014 | Loss: 0.6183 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss █▇▆▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▁▁▁▁
wandb:  test_acc ▂▁▄▃▄▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▆█▅▇▆▇███▇██▇▇█
wandb: train_acc ▁▄▄▄▄▅▆▆▅▆▇▆▆▆▇▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇█▇███
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.57733
wandb:  test_acc 0.71873
wandb: train_acc 0.70949
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/q0adqzn4
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_200858-q0adqzn4/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/worki

Seed 30 | Epoch 001 | Loss: 0.6906 | Train Acc: 0.5255 | Test Acc: 0.6345
Seed 30 | Epoch 002 | Loss: 0.6764 | Train Acc: 0.5786 | Test Acc: 0.5650
Seed 30 | Epoch 003 | Loss: 0.6748 | Train Acc: 0.5693 | Test Acc: 0.5280
Seed 30 | Epoch 004 | Loss: 0.6702 | Train Acc: 0.5718 | Test Acc: 0.6370
Seed 30 | Epoch 005 | Loss: 0.6572 | Train Acc: 0.6131 | Test Acc: 0.6443
Seed 30 | Epoch 006 | Loss: 0.6519 | Train Acc: 0.6058 | Test Acc: 0.6467
Seed 30 | Epoch 007 | Loss: 0.6460 | Train Acc: 0.6331 | Test Acc: 0.6594
Seed 30 | Epoch 008 | Loss: 0.6372 | Train Acc: 0.6399 | Test Acc: 0.6691
Seed 30 | Epoch 009 | Loss: 0.6326 | Train Acc: 0.6506 | Test Acc: 0.6720
Seed 30 | Epoch 010 | Loss: 0.6298 | Train Acc: 0.6550 | Test Acc: 0.6672
Seed 30 | Epoch 011 | Loss: 0.6241 | Train Acc: 0.6462 | Test Acc: 0.6900
Seed 30 | Epoch 012 | Loss: 0.6185 | Train Acc: 0.6564 | Test Acc: 0.6540
Seed 30 | Epoch 013 | Loss: 0.6166 | Train Acc: 0.6628 | Test Acc: 0.6774
Seed 30 | Epoch 014 | Loss: 0.6143 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁
wandb:  test_acc ▅▂▁▅▅▆▆▆▆▆▆▇▅▆▇▆▇▇▇█▅▆▇▇▇██▇█▇█▇▇█▇█▇▇▇█
wandb: train_acc ▁▃▃▃▄▅▅▅▆▅▆▆▆▇▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇█▇█▇▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.56476
wandb:  test_acc 0.71679
wandb: train_acc 0.72165
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal/runs/flkomue8
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28NCI1%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_201423-flkomue8/logs
